# Landsat-8 NDVI visualization

### Import required files

In [3]:
from IPython.display import Image
import ee, folium
import pandas as pd
import numpy as np
# from pylab import *
from matplotlib.pylab import rcParams

ee.Initialize()
%matplotlib inline

### Select area of interest and the collection to be used

In [5]:
# Polygon selected is a site near Yosemite
roi = ee.Geometry.Polygon([[-120.3066492132134, 37.68165381501888],
          [-120.3066492132134, 37.528794431827464],
          [-119.9331140569634, 37.528794431827464],
          [-119.9331140569634, 37.68165381501888]])
l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")

### Visualize region of interest

In [24]:
vis_params = {'bands':['B4','B3','B2'], 'max':0.3, 'min':0}
filtered = l8.filterDate('2016-01-01','2017-01-01').filterBounds(roi);

mapid = filtered.getMapId(vis_params)

map = folium.Map(location=[37.61, -120.12],zoom_start=9, height=500,width=700)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Landsat 8 ',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

### Function for returning NDVI

In [45]:
def retNDVI(image):
  return image.normalizedDifference(['B5', 'B4'])

### Visualize NDVI plot

In [35]:
ndvi = addNDVI(l8.min())

image = ndvi.clip(roi)
vis_ndvi = {'min':-1,'max':1,'size':'800',
           'region':roi}

mapid = image.getMapId(vis_ndvi)

map = folium.Map(location=[37.61, -120.12],zoom_start=11, height=500,width=700)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDVI',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

### Greenest pixel composite 

In [53]:
# Creating ndvi collection
ndvi_collection = l8.map(addNDVI)
# Selecting greenest pixels of collection
greenest = ndvi_collection.qualityMosaic('nd')

#Visualizing
greenest_vis = {'bands':'nd','min':0,'max':1}

mapid = greenest.getMapId(greenest_vis)
map = folium.Map(location=[37.61, -120.12],zoom_start=9, height=500,width=700)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Landsat 8 ',
  ).add_to(map)

map.add_child(folium.LayerControl())
map